In [4]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
df = hd.read_txn_data("777")
hd.inspect(df)

Time for read_txn_data                 : 2.36 seconds
(651,789, 18)


,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,female,id,latest_balance,postcode,tag_auto,yob
0,2012-02-01,777,400.00,<mdbremoved> - s/o,NaN,transfers,other_transfers,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688300,364.220001,wa1 4,transfers,1969.0
1,2012-02-01,777,3.03,aviva pa - d/d,aviva,spend,finance,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688299,364.220001,wa1 4,health insurance,1969.0


In [57]:
dfa = hd.read_analysis_data("777")
hd.inspect(dfa)

Time for read_analysis_data            : 0.47 seconds
(729, 27)


,user_id,month,date,active_accounts,age,annual_income,entropy,entropyz,female,has_sa_inflows,month_income,month_spend,reg_sa_inflows,sa_inflows,spend_communication,spend_finance,spend_hobbies,spend_household,spend_motor,spend_other_spend,spend_retail,spend_services,spend_travel,txn_count_ca,txn_count_sa,txns_count,txns_value
0,13777,11,2012-11-30,"[112648, 112650, 112652, 112651]",43.0,69899.824219,0.827491,0.081098,0.0,1,6366.030273,11073.820312,0,50076.0,0.0,1151.430054,25.000000,6983.815918,0.000000,1686.0,501.670013,403.660004,136.899994,85.0,14.0,100,317126.781250
1,13777,12,2012-12-31,"[112648, 112650, 112652, 112651, 112653]",43.0,69899.824219,0.853535,0.389008,0.0,1,5283.939941,7344.680176,0,3250.0,0.0,13.530000,94.989998,5677.649902,202.669998,1030.0,183.949997,141.889999,0.000000,52.0,8.0,65,34068.792969


In [59]:
dfa.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,729.0,354516.368999,147382.596599,13777.000000,316777.000000,393777.000000,430777.000000,5.537770e+05
month,729.0,6.427984,3.450687,1.000000,3.000000,6.000000,9.000000,1.200000e+01
age,729.0,38.458162,12.944459,19.000000,28.000000,37.000000,44.000000,7.100000e+01
annual_income,729.0,27682.949957,17111.518305,10535.050781,18128.519531,22883.554688,30535.039062,9.688784e+04
entropy,729.0,0.813192,0.076777,0.440339,0.770192,0.819963,0.863935,9.766147e-01
entropyz,729.0,-0.087953,0.907710,-4.496099,-0.596338,-0.007907,0.511965,1.844151e+00
female,729.0,0.441701,0.496931,0.000000,0.000000,0.000000,1.000000,1.000000e+00
has_sa_inflows,729.0,0.536351,0.499019,0.000000,0.000000,1.000000,1.000000,1.000000e+00
month_income,729.0,2261.288330,1426.108032,-0.000000,1453.540039,1918.660034,2771.600098,9.285477e+03
month_spend,729.0,2269.852295,2073.419434,208.640015,1123.580078,1740.650024,2568.259766,1.760786e+04


In [63]:
r = ag.month_spend(df)
r

Time for month_spend                   : 0.29 seconds


tag                        spend_finance  spend_services  spend_other_spend  \
user_id date       credit                                                     
777     2012-02-29 False      115.930000      103.690002          66.180000   
        2012-04-30 False       83.090004       11.990000         100.000000   
        2012-05-31 False      133.119995      138.979996         384.229980   
        2012-06-30 False      125.120003       96.650002         259.839996   
        2012-07-31 False      142.639999      133.779999         316.980011   
...                                  ...             ...                ...   
587777  2020-03-31 False    10114.980469     2659.570068         231.179993   
        2020-04-30 False             NaN      643.780029          15.080000   
        2020-05-31 False             NaN      600.570007         176.100006   
        2020-06-30 False             NaN      461.040009          86.160004   
        2020-07-31 False             NaN      557.539978         249.630005   

tag                        spend_household  spend_motor  spend_communication  \
user_id date       credit                                                      
777     2012-02-29 False         55.379997    91.479996            57.590000   
        2012-04-30 False          8.560000    51.180000                  NaN   
        2012-05-31 False        113.919998    44.849998            36.180000   
        2012-06-30 False         20.250000    78.139999            36.009998   
        2012-07-31 False         82.529999   181.709991            36.009998   
...                                    ...          ...                  ...   
587777  2020-03-31 False      51261.410156   143.290009            18.820000   
        2020-04-30 False      31283.830078     2.000000            23.480000   
        2020-05-31 False       1168.579956     6.650000            41.169998   
        2020-06-30 False       1035.910034   100.959999            31.160000   
        2020-07-31 False        356.850006     3.200000            29.170000   

tag                        spend_retail  spend_travel  spend_hobbies  
user_id date       credit                                             
777     2012-02-29 False            NaN           NaN            NaN  
        2012-04-30 False      29.990000           NaN            NaN  
        2012-05-31 False      90.680000     74.250000            NaN  
        2012-06-30 False            NaN           NaN            NaN  
        2012-07-31 False     162.460007           NaN            NaN  
...                                 ...           ...            ...  
587777  2020-03-31 False     272.100006     82.730003            NaN  
        2020-04-30 False     105.580002     10.930000            NaN  
        2020-05-31 False     279.070007     10.930000            NaN  
        2020-06-30 False     155.630005     36.529999            NaN  
        2020-07-31 False      79.730003     42.330002            NaN  

[7375 rows x 9 columns]

In [53]:
r.reg_sa_inflows.describe()

count    7578.000000
mean        0.261942
std         0.439721
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: reg_sa_inflows, dtype: float64

In [49]:
r.groupby("user_id").has_sa_inflows.rolling(window=12, min_periods=1).sum().ge(
    10
).astype(int).droplevel(0)

user_id  date      
777      2012-02-29    0
         2012-04-30    0
         2012-05-31    0
         2012-06-30    0
         2012-07-31    0
                      ..
587777   2020-03-31    0
         2020-04-30    0
         2020-05-31    0
         2020-06-30    0
         2020-07-31    0
Name: has_sa_inflows, Length: 7578, dtype: int64

In [16]:
kk

AttributeError: 'DataFrame' object has no attribute 'amount'